1. Linear Regression
2. Decision Tree
3. Random Forest
4. Support Vector Machine
5. Gradient Boosting
6. Neural Network
7. Lasso-Ridge
8. XGBoost
9. ElasticNet
10. AdaBoost

# Data Loading

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Load the CSV file into a pandas DataFrame
gdp = pd.read_excel('/content/gdrive/MyDrive/ADS Project/lagdp1222.xlsx')
#night = pd.read_excel('/content/gdrive/MyDrive/ADS Project/NightLightCounty_BINS.xlsx')
night = pd.read_csv('/content/gdrive/MyDrive/ADS Project/NightLightCounty_BINS.csv').drop('Unnamed: 0',axis=1)

# Data Preparation

In [ ]:
night.head()

,COUNTY_NAME,STATEFP,COUNTYFP,count_0_6,count_6_12,count_12_18,count_18_24,count_24_inf,count_nan
0,Ballard,21,7,3989,29,18,16,91,2710
1,Bourbon,21,17,4300,40,12,10,116,4032
2,Butler,21,31,6434,20,6,1,73,6138
3,Estill,21,65,3766,57,5,6,63,3453
4,Fleming,21,69,5319,23,13,5,48,5686


In [ ]:
# Define a dictionary that maps state names to STATEFP codes
state_codes = {
    'Alabama': '1',
    'Alaska': '2',
    'Arizona': '4',
    'Arkansas': '5',
    'California': '6',
    'Colorado': '8',
    'Connecticut': '9',
    'Delaware': '10',
    'District of Columbia': '11',
    'Florida': '12',
    'Georgia': '13',
    'Hawaii': '15',
    'Idaho': '16',
    'Illinois': '17',
    'Indiana': '18',
    'Iowa': '19',
    'Kansas': '20',
    'Kentucky': '21',
    'Louisiana': '22',
    'Maine': '23',
    'Maryland': '24',
    'Massachusetts': '25',
    'Michigan': '26',
    'Minnesota': '27',
    'Mississippi': '28',
    'Missouri': '29',
    'Montana': '30',
    'Nebraska': '31',
    'Nevada': '32',
    'New Hampshire': '33',
    'New Jersey': '34',
    'New Mexico': '35',
    'New York': '36',
    'North Carolina': '37',
    'North Dakota': '38',
    'Ohio': '39',
    'Oklahoma': '40',
    'Oregon': '41',
    'Pennsylvania': '42',
    'Rhode Island': '44',
    'South Carolina': '45',
    'South Dakota': '46',
    'Tennessee': '47',
    'Texas': '48',
    'Utah': '49',
    'Vermont': '50',
    'Virginia': '51',
    'Washington': '53',
    'West Virginia': '54',
    'Wisconsin': '55',
    'Wyoming': '56'
}

# Map the State column to the STATEFP codes and create a new column
gdp['STATEFP'] = gdp['State'].map(state_codes)

In [ ]:
gdp = gdp.rename(columns={'County': 'COUNTY_NAME'})

In [ ]:
gdp.head()

,COUNTY_NAME,State,GDP_2021,STATEFP
0,Autauga,Alabama,1502153,1
1,Baldwin,Alabama,7830237,1
2,Barbour,Alabama,709459,1
3,Bibb,Alabama,392249,1
4,Blount,Alabama,997835,1


In [ ]:
gdp['log_GDP_2021'] = np.log(pd.to_numeric(gdp['GDP_2021'], errors='coerce') + 1e-3)

In [ ]:
gdp.head()

,COUNTY_NAME,State,GDP_2021,STATEFP,log_GDP_2021
0,Autauga,Alabama,1502153,1,14.222410
1,Baldwin,Alabama,7830237,1,15.873503
2,Barbour,Alabama,709459,1,13.472258
3,Bibb,Alabama,392249,1,12.879652
4,Blount,Alabama,997835,1,13.813343


In [ ]:
# Convert the STATEFP column in both dataframes to the same data type
gdp['STATEFP'] = gdp['STATEFP'].astype(str)
night['STATEFP'] = night['STATEFP'].astype(str)

merged_df = pd.merge(gdp, night, on=['COUNTY_NAME', 'STATEFP'])

In [ ]:
merged_df

,COUNTY_NAME,State,GDP_2021,STATEFP,log_GDP_2021,COUNTYFP,count_0_6,count_6_12,count_12_18,count_18_24,count_24_inf,count_nan
0,Autauga,Alabama,1502153,1,14.222410,1,7990,211,64,36,369,3358
1,Baldwin,Alabama,7830237,1,15.873503,3,19824,1119,375,245,1999,18678
2,Barbour,Alabama,709459,1,13.472258,5,12429,144,42,32,206,8779
3,Bibb,Alabama,392249,1,12.879652,7,8701,124,32,13,146,4316
4,Blount,Alabama,997835,1,13.813343,9,8949,270,50,23,159,9749
...,...,...,...,...,...,...,...,...,...,...,...,...
3054,Sweetwater,Wyoming,3125840,56,14.955214,37,168173,190,78,61,775,18607
3055,Teton,Wyoming,2862327,56,14.867145,39,70441,33,17,4,67,12583
3056,Uinta,Wyoming,771851,56,13.556547,41,33092,65,32,18,251,764
3057,Washakie,Wyoming,346859,56,12.756674,43,37343,27,10,9,71,18407


In [ ]:
merged_df.columns

Index(['COUNTY_NAME', 'State', 'GDP_2021', 'STATEFP', 'log_GDP_2021',
       'COUNTYFP', 'count_0_6', 'count_6_12', 'count_12_18', 'count_18_24',
       'count_24_inf', 'count_nan'],
      dtype='object')

In [ ]:
merged_df.dropna(inplace=True)

In [ ]:
merged_df

,COUNTY_NAME,State,GDP_2021,STATEFP,log_GDP_2021,COUNTYFP,count_0_6,count_6_12,count_12_18,count_18_24,count_24_inf,count_nan
0,Autauga,Alabama,1502153,1,14.222410,1,7990,211,64,36,369,3358
1,Baldwin,Alabama,7830237,1,15.873503,3,19824,1119,375,245,1999,18678
2,Barbour,Alabama,709459,1,13.472258,5,12429,144,42,32,206,8779
3,Bibb,Alabama,392249,1,12.879652,7,8701,124,32,13,146,4316
4,Blount,Alabama,997835,1,13.813343,9,8949,270,50,23,159,9749
...,...,...,...,...,...,...,...,...,...,...,...,...
3054,Sweetwater,Wyoming,3125840,56,14.955214,37,168173,190,78,61,775,18607
3055,Teton,Wyoming,2862327,56,14.867145,39,70441,33,17,4,67,12583
3056,Uinta,Wyoming,771851,56,13.556547,41,33092,65,32,18,251,764
3057,Washakie,Wyoming,346859,56,12.756674,43,37343,27,10,9,71,18407


# **Data Modeling**

# Linear Regression

In [ ]:
# Define your dependent variable and independent variables
y = merged_df['GDP_2021'].astype(float)
cols = [i for i in merged_df.columns if 'count_' in i]
X = merged_df[cols].astype(float)

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Create and fit the model
model = sm.OLS(y, X).fit()

# Print the model summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               GDP_2021   R-squared:                       0.606
Model:                            OLS   Adj. R-squared:                  0.605
Method:                 Least Squares   F-statistic:                     780.9
Date:                Sat, 29 Apr 2023   Prob (F-statistic):               0.00
Time:                        01:34:09   Log-Likelihood:                -55313.
No. Observations:                3059   AIC:                         1.106e+05
Df Residuals:                    3052   BIC:                         1.107e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         8.963e+05   4.36e+05      2.057   

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Select the relevant columns from your dataframe as X (predictor variables)
X = merged_df.iloc[:,5:31]

# Set the GDP values as y (target variable)
y = merged_df['GDP_2021']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the linear regression model
lr = LinearRegression()

# Fit the model to the training data
lr.fit(X_train, y_train)

# Predict on the test data
y_pred = lr.predict(X_test)

# Evaluate the model using mean squared error
mse_lr = mean_squared_error(y_test, y_pred)
r2_lr = r2_score(y_test, y_pred)

# Print the mean squared error
print("Mean Squared Error:", mse_lr)
print("R-squared: ", r2_lr)

Mean Squared Error: 276561675238629.38
R-squared:  0.7313928307771607


# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Initialize the decision tree regressor
dt = DecisionTreeRegressor(max_depth=5, random_state=42)

# Fit the model on the training data
dt.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = dt.predict(X_test)

# Calculate the mean squared error of the predictions
mse_dt = mean_squared_error(y_test, y_pred)

# Calculate the R-squared score
r2_dt = r2_score(y_test, y_pred)

print("Mean Squared Error: ", mse_dt)
print("R-squared: ", r2_dt)

Mean Squared Error:  386753977636641.1
R-squared:  0.6243698949646039


# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Select the relevant columns from your dataframe as X (predictor variables)
X = merged_df.iloc[:,5:31]

# Set the GDP values as y (target variable)
y = merged_df['GDP_2021']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200, 500, 100],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5],
    'max_features': ['auto', 'sqrt']
}

# Create a random forest regressor object
rf = RandomForestRegressor()

# Create a GridSearchCV object and fit it to the data
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and their corresponding performance score
print('Best hyperparameters:', grid_search.best_params_)
print('Best score:', grid_search.best_score_)

Best hyperparameters: {'max_depth': 10, 'max_features': 'auto', 'min_samples_split': 5, 'n_estimators': 50}
Best score: 0.5044609363831645


In [ ]:
# Initialize the random forest regressor with the desired number of trees
rf = RandomForestRegressor(max_depth=5, max_features='auto', min_samples_split=2, n_estimators=100, random_state=42)

# Fit the model on the training data
rf.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = rf.predict(X_test)

# Calculate the mean squared error of the predictions
mse_rf = mean_squared_error(y_test, y_pred)

# calculate the R-squared score
r2_rf = r2_score(y_test, y_pred)

print("Mean Squared Error: ", mse_rf)
print("R-squared:", r2_rf)

Mean Squared Error:  492463203823016.5
R-squared: 0.521701092491668


# SVM

In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score

# Define the SVM model with hyperparameters
svm = SVR(kernel='linear', C=1.0, epsilon=0.1)
#svm = SVR(kernel='rbf', C=1, gamma='scale')

# Fit the model to the training data
svm.fit(X_train, y_train)

# Make predictions on the test data
y_pred = svm.predict(X_test)

# Calculate the mean squared error
mse_svm = mean_squared_error(y_test, y_pred)

# Calculate the r-squared
r2_svm = r2_score(y_test, y_pred)

# Print the results
print("Mean Squared Error:", mse_svm)
print("R-squared:", r2_svm)

Mean Squared Error: 458856710998166.5
R-squared: 0.5543409906171106


# Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

# Define the model
gbm = GradientBoostingRegressor(random_state=42)

# Define the range of hyperparameters to search
param_grid = {
    'n_estimators': [100, 500, 1000],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.5],
}

# Perform the grid search
grid_search = GridSearchCV(gbm, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and the corresponding MSE
print("Best hyperparameters:", grid_search.best_params_)
print("Best MSE:", -grid_search.best_score_)

Best hyperparameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 500}
Best MSE: 367207071635238.2


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Define the gradient boosting model
gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Fit the model to the training data
gbr.fit(X_train, y_train)

# Predict the output of the test data
y_pred = gbr.predict(X_test)

# Evaluate the model performance
mse_gb = mean_squared_error(y_test, y_pred)
r2_gb = r2_score(y_test, y_pred)

print("MSE: ", mse_gb)
print("R-squared: ", r2_gb)

MSE:  549233783642901.56
R-squared:  0.46656335611730915


# Neural Network

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Define the model architecture
model = MLPRegressor(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', max_iter=1000, random_state=42)

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Evaluate the model performance
mse_neural_network = mean_squared_error(y_test, y_pred)
r2_neural_network = r2_score(y_test, y_pred)

print("MSE: ", mse_neural_network)
print("R-squared: ", r2_neural_network)

MSE:  200247945076521.66
R-squared:  0.8055116146397203


# Lasso-Ridge 

In [ ]:
from sklearn.linear_model import Lasso, Ridge
import numpy as np

# Create Lasso-Ridge model
mse_lasso_ridge = []
alphas = np.linspace(0.001, 10, 100)
for alpha in alphas:
    lasso = Lasso(alpha=alpha)
    ridge = Ridge(alpha=alpha)
    lasso.fit(X_train, y_train)
    ridge.fit(X_train, y_train)
    y_pred_lasso = lasso.predict(X_test)
    y_pred_ridge = ridge.predict(X_test)
    y_pred_lasso_ridge = 0.5 * (y_pred_lasso + y_pred_ridge)
    mse = mean_squared_error(y_test, y_pred_lasso_ridge)
    mse_lasso_ridge.append(mse)

# Print best alpha value
best_alpha = alphas[np.argmin(mse_lasso_ridge)]
print("Best alpha value:", best_alpha)

Best alpha value: 0.001


In [ ]:
from sklearn.linear_model import Lasso, Ridge
from sklearn.metrics import mean_squared_error, r2_score

# Create Lasso-Ridge model with best alpha value
alpha = 10
lasso = Lasso(alpha=alpha)
ridge = Ridge(alpha=alpha)
lasso.fit(X_train, y_train)
ridge.fit(X_train, y_train)
y_pred_lasso = lasso.predict(X_test)
y_pred_ridge = ridge.predict(X_test)
y_pred_lasso_ridge = 0.5 * (y_pred_lasso + y_pred_ridge)

# Compute MSE and R-squared
mse_lasso_ridge = mean_squared_error(y_test, y_pred_lasso_ridge)
r2_lasso_ridge = r2_score(y_test, y_pred_lasso_ridge)

# Print Lasso-Ridge model performance
print("Lasso-Ridge MSE:", mse_lasso_ridge)
print("Lasso-Ridge R-squared:", r2_lasso_ridge)

Lasso-Ridge MSE: 276561722775366.0
Lasso-Ridge R-squared: 0.7313927846076824


# XGBoost

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score

X_train_xgb = xgb.DMatrix(X_train,label = y_train)
X_test_xgb = xgb.DMatrix(X_test,label = y_test)

param= {
    'max_depth': 7,
    'eta': 0.01,
    'objective': 'reg:squarederror'
}
epochs = 1000
model = xgb.train(param,X_train_xgb,epochs)

y_pred = model.predict(X_test_xgb)

mse_xgb = mean_squared_error(y_test, y_pred)
r2_xgb = r2_score(y_test, y_pred)

print("MSE: ", mse_xgb)
print("R-squared: ", r2_xgb)

MSE:  737208041361891.6
R-squared:  0.28399564058298443


# ElasticNet

In [ ]:
from sklearn.linear_model import ElasticNet

# Create ElasticNet model
elasticnet = ElasticNet(alpha=0.1, l1_ratio=0.5)

# Train ElasticNet model on the training data
elasticnet.fit(X_train, y_train)

# Make predictions using ElasticNet model on the test data
y_pred_elasticnet = elasticnet.predict(X_test)

# Evaluate ElasticNet model performance using mean squared error
mse_elasticnet = mean_squared_error(y_test, y_pred_elasticnet)
r2_elasticnet = r2_score(y_test, y_pred_elasticnet)

# Print ElasticNet model performance
print("ElasticNet MSE: ", mse_elasticnet)
print("ElasticNet R-squared: ", r2_elasticnet)

ElasticNet MSE:  276562784430397.2
ElasticNet R-squared:  0.7313917534881235


# AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Define base estimator and AdaBoost regressor
base_estimator = DecisionTreeRegressor(max_depth=4)
adaboost = AdaBoostRegressor(base_estimator=base_estimator, n_estimators=100, learning_rate=0.1)

# Train the model
adaboost.fit(X_train, y_train)

# Make predictions
y_pred = adaboost.predict(X_test)

# Calculate mean squared error and R-squared
mse_adaboost = mean_squared_error(y_test, y_pred)
r2_adaboost = r2_score(y_test, y_pred)

# Print results
print("MSE:", mse_adaboost)
print("R-squared:", r2_adaboost)

MSE: 1040410766883897.8
R-squared: -0.010486325267285768


# Comparing different models

In [ ]:
# Create a list of models and their corresponding MSE and R-squared values
models = [
    {'Model': 'Linear Regression', 'MSE': mse_lr, 'R-squared': r2_lr},
    {'Model': 'Decision Tree', 'MSE': mse_dt, 'R-squared': r2_dt},
    {'Model': 'Random Forest', 'MSE': mse_rf, 'R-squared': r2_rf},
    {'Model': 'Support Vector Machine', 'MSE': mse_svm, 'R-squared': r2_svm},
    {'Model': 'Neural Network', 'MSE': mse_neural_network, 'R-squared': r2_neural_network},
    {'Model': 'Lasso-Ridge', 'MSE': mse_lasso_ridge, 'R-squared': r2_lasso_ridge},
    {'Model': 'Gradient Boosting', 'MSE': mse_gb, 'R-squared': r2_gb},
    {'Model': 'XGBoost', 'MSE': mse_xgb, 'R-squared': r2_xgb},
    {'Model': 'ElasticNet', 'MSE': mse_elasticnet, 'R-squared': r2_elasticnet},
    {'Model': 'AdaBoost', 'MSE': mse_adaboost, 'R-squared': r2_adaboost}
]

# Create a DataFrame from the models list
df_table = pd.DataFrame(models)

In [ ]:
df_table

,Model,MSE,R-squared
0,Linear Regression,2.765617e+14,0.731393
1,Decision Tree,3.867540e+14,0.624370
2,Random Forest,4.924632e+14,0.521701
3,Support Vector Machine,4.588567e+14,0.554341
4,Neural Network,2.002479e+14,0.805512
5,Lasso-Ridge,2.765617e+14,0.731393
6,Gradient Boosting,5.492338e+14,0.466563
7,XGBoost,7.372080e+14,0.283996
8,ElasticNet,2.765628e+14,0.731392
9,AdaBoost,1.040411e+15,-0.010486
